In [1]:
import torch
import numpy as np 

### Задача 1
На вход идет изображение 28х28. Применить свертку так, чтобы в результате получить 5 карт характеристик размером
- 28х28
- 26х26
- 14х14

In [15]:
t = torch.randn(1,1,28,28)
# разменость 0 - batch, т.е. мы подаем пакет, который содержит одну картинку
# размерность 1 - это глубина картинки (количество каналов, например rgb)
# последние 2 размерности это высота и ширина картинки
t2 = torch.randn(2,3,28,28) # здесь задается пакет из 2-х картинок размера 28х28х3


In [14]:
conv1 = torch.nn.Conv2d(1,5,kernel_size=3,stride=1,padding=1)
# 1 - количество входных каналов, 5 - количество выходных каналов (количество сверток), kernel_size - размер свертки, padding - на сколько свертка выходит за пределы исходной картинки
out1 = conv1(t)
print(out1.shape) # получили 5 матриц (каналов) размером 28х28
conv2 = torch.nn.Conv2d(1,5,kernel_size=3,stride=1,padding=0)
out2 = conv2(t)
print(out2.shape)
conv3 = torch.nn.Conv2d(1,5,kernel_size=3,stride=2,padding=1)
out3 = conv3(t)
print(out3.shape)

torch.Size([1, 5, 28, 28])
torch.Size([1, 5, 26, 26])
torch.Size([1, 5, 14, 14])


### Задача 2
На вход идет изображение 28х28. Реализовать следующую архитектуру: входное изображение разбивается на 3 потока. 
- Первый поток обрабатывается 4-я свертками 1х1, polling с шагом 2
- Второй поток обрабатывается 3-я свертками 3х3, polling с шагом 2
- Третий поток обрабатывается 2-я свертками 5х5, polling с шагом 2

Выходы потоков необходимо конкатенировать (объединить) и получить 9 карт характеристик

In [22]:
t = torch.randn(1,1,28,28)
conv1 = torch.nn.Conv2d(1,4,kernel_size=1)
conv2 = torch.nn.Conv2d(1,3,kernel_size=3,padding=1)
conv3 = torch.nn.Conv2d(1,2,kernel_size=5,padding=2)
mp = torch.nn.MaxPool2d(2)

out1 = mp(conv1(t))
out2 = mp(conv2(t))
out3 = mp(conv3(t))
out = torch.cat((out1,out2,out3),dim=1)
out.shape

torch.Size([1, 9, 14, 14])

### Задача 3
На вход идет изображение 28х28. Реализовать следующую архитектуру: входное изображение разбивается на 3 потока. 
- Первый поток обрабатывается 4-я свертками 1х1, polling с шагом 2
- Второй поток обрабатывается 3-я свертками 3х3, polling с шагом 2
- Третий поток обрабатывается 2-я свертками 5х5, polling с шагом 2

 Организовать каждый поток в виде блока  sequential. Выходы потоков необходимо конкатенировать (объединить) и получить 9 карт характеристик

In [23]:
t = torch.randn(1,1,28,28)
seq1 = torch.nn.Sequential(torch.nn.Conv2d(1,4,kernel_size=1), torch.nn.MaxPool2d(2))
seq2 = torch.nn.Sequential(torch.nn.Conv2d(1,3,kernel_size=3,padding=1), torch.nn.MaxPool2d(2))
seq3 = torch.nn.Sequential(torch.nn.Conv2d(1,2,kernel_size=5,padding=2), torch.nn.MaxPool2d(2))


out1 = seq1(t)
out2 = seq2(t)
out3 = seq3(t)
out = torch.cat((out1,out2,out3),dim=1)
out.shape

torch.Size([1, 9, 14, 14])

### Задача 4
На вход идет изображение 28х28. Реализовать следующую архитектуру: входное изображение разбивается на 3 потока. 
- Первый поток обрабатывается 4-я свертками 1х1, polling с шагом 2
- Второй поток обрабатывается 3-я свертками 3х3, polling с шагом 2
- Третий поток обрабатывается 2-я свертками 5х5, polling с шагом 2

 Организовать каждый поток в виде блока  sequential. Выходы потоков необходимо конкатенировать (объединить) и получить 9 карт характеристик.
 - Применить к результату нелинейность ReLU
 - Выход ReLU подать на полносвязный слой, на выходе которого 10 значений (10 классов)

In [25]:
t = torch.randn(1,1,28,28)
seq1 = torch.nn.Sequential(torch.nn.Conv2d(1,4,kernel_size=1), torch.nn.MaxPool2d(2))
seq2 = torch.nn.Sequential(torch.nn.Conv2d(1,3,kernel_size=3,padding=1), torch.nn.MaxPool2d(2))
seq3 = torch.nn.Sequential(torch.nn.Conv2d(1,2,kernel_size=5,padding=2), torch.nn.MaxPool2d(2))
relu = torch.nn.ReLU()

out1 = seq1(t)
out2 = seq2(t)
out3 = seq3(t)
out = relu(torch.cat((out1,out2,out3),dim=1))
out.shape

torch.Size([1, 9, 14, 14])

### Задача 5
Организовать задачу 4 используя классы (наследуя от nn.Module)

In [26]:
class convNet(torch.nn.Module):
    def __init__(self):
        super(convNet,self).__init__()
        self.seq1 = torch.nn.Sequential(torch.nn.Conv2d(1,4,kernel_size=1), torch.nn.MaxPool2d(2))
        self.seq2 = torch.nn.Sequential(torch.nn.Conv2d(1,3,kernel_size=3,padding=1), torch.nn.MaxPool2d(2))
        self.seq3 = torch.nn.Sequential(torch.nn.Conv2d(1,2,kernel_size=5,padding=2), torch.nn.MaxPool2d(2))
        self.relu = torch.nn.ReLU()
    def forward(self,x):
        x1 = self.seq1(x)
        x2 = self.seq2(x)
        x3 = self.seq3(x)
        x = torch.cat((x1,x2,x3),dim=1)
        x = self.relu(x)
        return x


In [27]:
model = convNet()
out = model(t)
t.shape

torch.Size([1, 1, 28, 28])